In [1]:
import pysam
import pandas as pd

transcript = "/Users/Miko/Desktop/SP18/BIMM 182/Project/refseq.genes.ncbi37"
chr_name = 'chr1'

In [2]:
codonMap = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L",
            "TCT":"S", "TCC":"s", "TCA":"S", "TCG":"S",
            "TAT":"Y", "TAC":"Y", "TAA":"STOP", "TAG":"STOP",
            "TGT":"C", "TGC":"C", "TGA":"STOP", "TGG":"W",
            "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
            "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P",
            "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
            "CGT":"R", "CGC":"R", "CGA":"R", "CGG":"R",
            "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M",
            "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
            "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K",
            "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R",
            "GTT":"V", "GTC":"V", "GTA":"V", "GTG":"V",
            "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A",
            "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
            "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

stop_codon = ["TAA", "TAG", "TGA"]

In [3]:
df_transcript = pd.read_table(transcript)
df_transcript.head(10)

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,0,NM_032291,chr1,+,66999824,67210768,67000041,67208778,25,"66999824,67091529,67098752,67101626,67105459,6...","67000051,67091593,67098777,67101698,67105516,6...",0,SGIP1,cmpl,cmpl,"0,1,2,0,0,0,1,0,0,0,1,2,1,1,1,1,0,1,1,2,2,0,2,..."
1,1,NM_052998,chr1,+,33546713,33585995,33547850,33585783,12,"33546713,33546988,33547201,33547778,33549554,3...","33546895,33547109,33547413,33547955,33549728,3...",0,ADC,cmpl,cmpl,"-1,-1,-1,0,0,0,2,2,0,1,0,2,"
2,1,NM_001145278,chr1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767270,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"
3,1,NM_001145277,chr1,+,16767166,16786584,16767256,16785491,7,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,"
4,1,NM_001080397,chr1,+,8384389,8404227,8384389,8404073,8,"8384389,8385357,8385877,8390268,8395496,839787...","8384786,8385450,8386102,8390996,8395650,839805...",0,SLC45A1,cmpl,cmpl,"0,1,1,1,0,1,1,0,"
5,1,NM_018090,chr1,+,16767166,16786584,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",0,NECAP2,cmpl,cmpl,"0,2,1,1,2,0,1,2,"
6,1,NM_013943,chr1,+,25071759,25170815,25072044,25167428,6,"25071759,25124232,25140584,25153500,25166350,2...","25072116,25124342,25140710,25153607,25166532,2...",0,CLIC4,cmpl,cmpl,"0,0,2,2,1,0,"
7,1,NM_032785,chr1,-,48998526,50489626,48999844,50489468,14,"48998526,49000561,49005313,49052675,49056504,4...","48999965,49000588,49005410,49052838,49056657,4...",0,AGBL4,cmpl,cmpl,"2,2,1,0,0,2,1,1,0,2,0,1,1,0,"
8,2,NM_001195684,chr1,-,92145899,92371559,92149295,92327088,18,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...",0,TGFBR3,cmpl,cmpl,"1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,-1,"
9,2,NM_001195683,chr1,-,92145899,92351836,92149295,92327088,17,"92145899,92161228,92163645,92174219,92177799,9...","92149414,92161336,92163687,92174340,92178099,9...",0,TGFBR3,cmpl,cmpl,"1,1,1,0,0,0,0,0,1,0,2,1,0,0,1,0,-1,"


In [6]:
df_chr = df_transcript[df_transcript['chrom'] == chr_name]
genes_in_chr = df_chr['name'].tolist()

In [7]:
# extract exons for all genes of the chromosome
def extract_exons(chr_name):
    
    # a list that contains info for all the genes
    exon_list_all_genes = []      
    # iterate over all genes of chr  
    for i in range(len(genes_in_chr)):
        # get a list of all the exon start and end positions for this gene
        exon_list = []         
        starts = df_chr.loc[i, 'exonStarts'][:-1].split(',')
        ends = df_chr.loc[i, 'exonEnds'][:-1].split(',')

        for s, e in zip(starts, ends):  
            exon_list.append([s,e])

        frames = df_chr.loc[i, 'exonFrames'][:-1].split(',')
        print(frames)

        # get the indices of exons to remove (untranslated, UTR)
        exon_to_remove = []
        for index, frame in list(enumerate(frames)):
            if int(frame) == -1:
                exon_to_remove.append(index)

        # EXONS that are translated
        exon_list = [i for j, i in enumerate(exon_list) if j not in exon_to_remove]
        exon_list_all_genes.append(exon_list)
        
    return exon_list_all_genes


In [8]:
extract_exons(chr_name)

['0', '1', '2', '0', '0', '0', '1', '0', '0', '0', '1', '2', '1', '1', '1', '1', '0', '1', '1', '2', '2', '0', '2', '1', '1']
['-1', '-1', '-1', '0', '0', '0', '2', '2', '0', '1', '0', '2']
['0', '2', '1', '1', '2', '0', '1', '2']
['0', '2', '1', '1', '2', '0', '1']
['0', '1', '1', '1', '0', '1', '1', '0']
['0', '2', '1', '1', '2', '0', '1', '2']
['0', '0', '2', '2', '1', '0']
['2', '2', '1', '0', '0', '2', '1', '1', '0', '2', '0', '1', '1', '0']
['1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '2', '1', '0', '0', '1', '0', '-1', '-1']
['1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '2', '1', '0', '0', '1', '0', '-1']
['-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1']
['0', '0', '0', '0', '1', '0', '1', '2', '1', '0', '0']
['1', '1', '1', '0', '0', '0', '0', '0', '1', '0', '2', '1', '0', '0', '1', '0', '-1']
['-1', '0', '1', '0', '2', '0']
['0', '0', '1', '2', '1', '1', '0', '2', '2', '1', '0', '2', '0', '0', '0', '0', '1',

['0', '1', '1', '0', '1', '0', '0', '1']
['1', '0', '0', '0', '0', '1', '1', '2', '0', '2', '1', '0', '0', '2', '0', '2', '0', '2', '1', '2', '0', '0', '0', '2', '0', '0', '2', '2', '0', '-1']
['-1', '0', '1', '1', '0', '1', '1', '2', '0', '0', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1']
['0', '0', '1', '2', '0', '0', '1', '1', '0', '2', '0', '2', '0', '0', '0', '0', '0', '0', '0', '2', '2', '0', '0', '2', '0', '0', '2', '1', '0', '2', '1', '0', '2', '1', '2', '1', '2', '0', '1', '1', '2', '0', '2', '0']
['0', '1', '0', '0', '0', '1', '0', '1', '2', '1', '2', '0', '2', '1', '2', '1', '0']
['0', '1', '0', '0', '1', '0', '0', '0', '2', '1', '0', '0', '2', '1', '2', '1', '0', '0', '0', '2', '1', '1', '1', '0']
['1', '0', '2', '1', '0', '-1']
['1', '0', '2', '1', '0', '-1']
['1', '0', '2', '1', '0', '-1']
['0', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '1']
['-1', '0', '1', '1', '1', '1', '1', '1', '2']
['0', '2', '2', '2', '2

['-1', '0']
['1', '1', '0', '1', '1', '1', '1', '2', '2', '1', '0', '-1', '-1']
['1', '1', '0', '1', '1', '1', '1', '2', '2', '1', '0', '-1', '-1']
['1', '1', '1', '0', '-1']
['1', '1', '1', '0', '-1']
['1', '1', '1', '0', '-1']
['1', '1', '0', '1', '1', '1', '1', '2', '2', '1', '0', '-1', '-1']
['0', '2', '0', '1', '0', '1', '0']
['0', '0', '2', '0', '2', '0', '0']
['-1']
['0', '1', '2', '0', '0', '2', '0', '1', '0']
['0', '0', '2', '0', '0', '2', '0', '0']
['-1', '-1', '-1', '0', '0', '1', '0']
['0', '0', '2', '2', '0', '0', '2', '0', '0']
['0', '0', '2', '2', '0', '0', '2', '2', '0', '0']
['-1', '-1', '-1', '-1', '0', '0']
['0', '0', '2', '0', '0', '2', '2', '0', '0']
['0', '1', '1', '1', '0', '1']
['0', '2', '1']
['0', '1', '2', '1', '1', '0', '1', '0']
['0', '1', '1', '-1']
['2', '0', '0', '0', '0', '0', '0', '1', '0']
['0', '1', '1', '1', '-1']
['0', '1', '2', '1', '1', '0', '1']
['-1']
['0', '0', '2']
['0', '-1', '-1']
['0', '-1']
['0', '-1', '-1']
['0', '0']
['0', '0', '2']
['0

['-1']
['1', '0', '0']
['-1', '-1']
['0', '0', '1', '1', '0']
['1', '2', '2', '2', '2', '0', '1', '2', '2', '0', '0']
['1', '2', '2', '2', '2', '0', '1', '2', '2', '0']
['0', '0', '2', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '-1']
['0', '0', '2', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '-1']
['0', '0', '2', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '-1']
['0', '0', '2', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '-1']
['0', '0', '0', '1', '2', '0', '0', '0', '1', '0', '0', '1', '1', '1']
['-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1']
['0', '0', '1', '1', '0', '0', '0', '0', '0', '2', '0']
['0', '0', '1', '1', '0', '0', '0', '2', '0']
['0', '0', '1', '1', '0', '0', '0', '0', '2', '0']
['-1', '0', '1', '0', '0', '0', '2', '0']
['1', '0', '0', '0']
['0', '1', '0', '0', '0']
['-1', '-1', '-1']
['-1', '-1', '-1']
['-1', '-1', '-1']
['-1', '-1', '-1'

['0', '2', '1', '0']
['0', '1', '1', '1', '0', '0', '1', '2', '1', '0']
['0', '0', '0', '0', '0', '2', '1', '0']
['2', '0', '1', '2', '0']
['-1', '0', '0', '1', '0', '0', '2', '2', '0', '1']
['-1', '0', '0', '1', '0', '0', '2', '2', '0', '1']
['-1', '0', '0', '1', '0', '0', '2', '2', '0', '1']
['-1', '0', '0', '1', '0', '0', '2', '2', '0', '1']
['0', '0', '1', '1']
['0', '1', '0', '-1', '-1']
['1', '2', '1', '0', '0', '2', '0', '0', '1', '1', '0', '0', '0', '2', '0', '1', '1', '2', '0', '2', '0', '2', '0', '1', '0', '2', '0', '-1']
['1', '0', '0', '1', '2', '0', '1', '1', '2', '2', '2', '0', '0', '0']
['-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1']
['-1', '0', '0', '1', '0', '0', '2', '2', '0', '1']
['-1', '0', '0', '0', '0', '1', '1', '2', '1', '0', '2', '0', '1', '0', '2', '0', '1', '2', '0', '1', '0', '0', '1', '0', '1', '0', '0']
['-1']
['0', '2', '1', '0', '1', '0', '1']
['2', '0', '1', '0', '0']
['-1', '0', '0', '0', '0', '1', '1', '2', '1', '0', '2', '0', '1', '0', '2', '

['0', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['2', '0', '0', '1', '0', '0', '0', '0', '2', '0', '0', '0', '1', '0', '0', '1', '2', '0', '1', '0', '1', '1', '1', '1', '0', '0', '0']
['2', '0', '0', '1', '0', '0', '0', '0', '2', '0', '0', '0', '0', '1', '0', '0', '1', '2', '0', '1', '0', '1', '1', '1', '1', '0', '0', '0']
['2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0']
['1', '2', '0']
['0', '1', '0', '2', '1', '0']
['0', '0', '2', '1', '0']
['0', '0', '0']
['-1', '2', '1', '0', '-1']
['0', '1', '0', '-1']
['-1', '0', '1']
['-1']
['-1']
['0']
['-1', '-1', '-1', '-1', '-1']
['0', '-1']
['0', '0', '0', '0', '0', '0', '0', '-1']
['-1', '0', '1', '0', '0', '1', '1', '2', '2', '0']
['0']
['0', '2', '1', '0', '0', '1', '1', '2', '2', '0']
['2', '0', '0', '2', '2', '1', '2', '2', '2', '0', '2', '1', '0', '0', '0', '2', '0', '0', '0', '1', '1', '0', '2', '2', '1', '0', '2', '0', '0', '0', '0', '0', '2', '0']
['2', '0', '0', '2', '2', '1', '2', '2', '2', '0', '2', '1', '0', '0',

[[['66999824', '67000051'],
  ['67091529', '67091593'],
  ['67098752', '67098777'],
  ['67101626', '67101698'],
  ['67105459', '67105516'],
  ['67108492', '67108547'],
  ['67109226', '67109402'],
  ['67126195', '67126207'],
  ['67133212', '67133224'],
  ['67136677', '67136702'],
  ['67137626', '67137678'],
  ['67138963', '67139049'],
  ['67142686', '67142779'],
  ['67145360', '67145435'],
  ['67147551', '67148052'],
  ['67154830', '67154958'],
  ['67155872', '67155999'],
  ['67161116', '67161176'],
  ['67184976', '67185088'],
  ['67194946', '67195102'],
  ['67199430', '67199563'],
  ['67205017', '67205220'],
  ['67206340', '67206405'],
  ['67206954', '67207119'],
  ['67208755', '67210768']],
 [['33547778', '33547955'],
  ['33549554', '33549728'],
  ['33557650', '33557823'],
  ['33558882', '33559017'],
  ['33560148', '33560314'],
  ['33562307', '33562470'],
  ['33563667', '33563780'],
  ['33583502', '33583717'],
  ['33585644', '33585995']],
 [['16767166', '16767270'],
  ['16770126', '16

In [ ]:
# extract the sequence for all the genes in this chromosome, and concatenate
# return a list of sequences, each corresponds to a gene
def extract_seq_concat(chr_name, exon_list_all_genes):
    seq_list = []
    fasta = pysam.FastaFile('/Users/Miko/Desktop/chromFa/' + chr_name +'.fa')
    # for each gene
    for index, exon_list in list(enumerate(exon_list_all_genes)):
    
        seq = ''
        for exon in exon_list:
            start = exon[0]
            end = exon[1]
            seq += fasta.fetch('', int(start), int(end), chr_name)
        
        # reverse complement if necessary
        strand = df_chr.loc[index, 'strand']
        if strand == '-':
            seq = reverse_complement(seq)
        
        seq = seq.upper()   # capitalized
        seq_list.append(seq)
        
    return seq_list
    

In [1]:
# extract the sequence for all the genes in this chromosome (without concat)
# return a list of sequences, each corresponds to a gene
def extract_seq(chr_name, exon_list_all_genes):
    seq_list_all_genes = []
    fasta = pysam.FastaFile('/Users/Miko/Desktop/chromFa/' + chr_name +'.fa')
    
    # for each gene
    for index, exon_list in list(enumerate(exon_list_all_genes)): 
        seq_list = []       
        for exon in exon_list:
            exon_seq = ''
            start = exon[0]
            end = exon[1]
            exon_seq = fasta.fetch('', int(start), int(end), chr_name).upper()
            
            # reverse complement if necessary
            strand = df_chr.loc[index, 'strand']
            if strand == '-':
                exon_seq = reverse_complement(exon_seq)
            
            seq_list.append(exon_seq)
             
        #seq = seq.upper()
        seq_list_all_genes.append(seq_list)
        
    return seq_list_all_genes

In [2]:
# return a reverse complement sequence
def reverse_complement(sequence):
    
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

    reverse_complement = "".join(complement.get(base, base) for base in reversed(sequence))
    return reverse_complement

In [ ]:
# Suppose four cases of mutation  
# frameshift = ['+1', '+2'] :
# +1: insert 1 or delete 2; 
# +2: insert 2 or delete 1
 
def rescue(gene, exon_list, exon_frames): 

    for exon_index, exon in list(enumerate(exon_list)):
        exon_len = len(exon)
            
        # get the lengths of previous and next exons (if they exist)
        if exon_index != 0:
            prev_len = len(exon_list[exon_index-1])
        else: # first exon
            prev_len = 0
            
        if exon_index != len(exon_list)-1:
            next_len = len(exon_list[exon_index+1])
        else: # last exon
            next_len = 0
        
        #print(prev_len)
        #print(next_len)
        
        # TODO: Run this when insert_base is A,C,G,T
        insert_one(gene, exon_list, exon_index, exon, exon_len, prev_len, next_len, exon_frames, insert_base)
        insert_two(gene, exon_list, exon_index, exon, exon_len, prev_len, next_len, exon_frames, insert_base)
        

In [ ]:
###### CASE 1: INSERT 1 BASE        
# Define insertion as inserting before that mutate_index
# PARAMETERS:
# exon_index: exon that has the mutation 
# insert_base: specify the mutation - which base gets inserted
# This function prints: 
# gene_name, mutated_exon_index, mutation_position, frameshift, skipped_exon_index

def insert_one(gene, exon_list, exon_index, exon, exon_len, prev_len, next_len, exon_frames, insert_base):

        spliced_exon = ''   # keep track of the deleted exon
        
        # can't rescue
        if (prev_len % 3 != 1) and (next_len % 3 != 1): 
            return  
        else:
            # insert at a given position (inserting before that mutate_index)
            for mutate_index in range(exon_len):

                if prev_len % 3 == 1:
                    # Keep track of the skipped exon (previous exon)
                    spliced_exon = exon_list[exon_index-1]

                    # TODO: CHANGE TO FUNCTION CALL
                    # no stop codon created
                    stop_codon_1 = False
                    if stop_codon_1 == False:
                        print(gene, exon_index, mutate_index, '+1', exon_index-1)
                        
                if next_len % 3 == 1:
                    # Keep track of the skipped exon (next exon)
                    spliced_exon = exon_list[exon_index+1]

                    # no stop codon created
                    stop_codon_2 = False
                    if stop_codon_2 == False:
                        print(gene, exon_index, mutate_index, '+1', exon_index+1)
                    else:
                        return   # can't rescue

In [ ]:
###### CASE 2: INSERT 2 BASES     
# Define insertion as inserting 2 bases before that mutate_index
# Prints: 
# gene_name, mutated_exon_index, mutation_position, frameshift, skipped_exon_index

def insert_two(gene, exon_list, exon_index, exon, exon_len, prev_len, next_len, exon_frames, insert_base):

        spliced_exon = ''   # keep track of the deleted exon
        
        # can't rescue
        if (prev_len % 3 != 2) and (next_len % 3 != 2): 
            return  
        else:
            # insert at a given position (inserting before that mutate_index)
            for mutate_index in range(exon_len):

                if prev_len % 3 == 2:
                    # Keep track of the skipped exon (previous exon)
                    spliced_exon = exon_list[exon_index-1]

                    # no stop codon created
                    stop_codon_1 = False
                    if stop_codon_1 == False:
                        print(gene, exon_index, mutate_index, '+2', exon_index-1)
                        
                if next_len % 3 == 2:
                    # Keep track of the skipped exon (next exon)
                    spliced_exon = exon_list[exon_index+1]

                    # no stop codon created
                    stop_codon_2 = False
                    if stop_codon_2 == False:
                        print(gene, exon_index, mutate_index, '+2', exon_index+1)
                    else:
                        return   # can't rescue

In [ ]:
# Each line containing: 
# gene_name, mutated_exon_index, mutation_position, frameshift, skipped_exon_index
rescue('gene1', ['AA', 'TTAC', 'TTA'], ['0', '1', '2'])
# FRAMES:   frames = df_chr.loc[i, 'exonFrames'][:-1].split(',')

In [17]:
# Find the range of sequence that might be changed to new codons due to splicing
def find_frames(exon_list, spliced_exon_index, exon_index, mutate_index, exon_frames, insert_base):
    
    seq_to_check = ''
    # the key tuple indicates the remainder and frame, the value is the number of bases remained
    dict_remainder_bases = {(1,0): 1, (1,1): 0, (1,2): 2,
                            (2,0): 2, (2,1): 1, (2,2): 0,
                            (0,0): 0, (0,1): 2, (0,2): 1} 
    
    # skip the previous exon:
    if spliced_exon_index < exon_index:
        
        # GET the remained bases on the exon before the spliced one 
        if spliced_exon_index == 0:
            remained_bases = ''
        else:
            len_prev_sp_exon = len(exon_list[spliced_exon_index-1])
            remainder = len_prev_sp_exon % 3
            frame_prev_sp_exon = int(exon_frames[spliced_exon_index-1]) # frame of that exon 

            marker = (remainder, frame_prev_sp_exon)
            print(marker)
            num_remained_bases = dict_remainder_bases[marker]  # number of bases remained from the exon before the spliced one
            print(num_remained_bases)
            # nothing remained
            if num_remained_bases == 0:
                remained_bases = ''
            else:
                remained_bases = exon_list[spliced_exon_index-1][(-num_remained_bases):]
        
        #print(remained_bases)

        # GET the bases on the mutated exon (up to the point of insertion)
        mutated_bases = exon_list[exon_index][:mutate_index] + insert_base
        #print(mutated_bases)
        
        seq_to_check = remained_bases + mutated_bases
        print(seq_to_check)
        return seq_to_check    

In [19]:
#find_frames(['AA', 'TTAC', 'TTA'], 1, 2, 0, ['0', '1', '2'])
# exon_list, spliced_exon_index, exon_index, mutate_index, exon_frames
find_frames(['AAAC', 'TTACT', 'TTAT', 'TT'], 2, 3, 1, ['2', '1', '2', '1'], 'ATT')


(2, 1)
1
T
TATT
TTATT


In [ ]:
## CHECK if a stop codon will be created
def stop_codon(exon_list, spliced_exon_index, exon_index, mutate_index, exon_frames, insert_base):
        
    seq_to_check = find_frames(exon_list, spliced_exon_index, exon_index, mutate_index, exon_frames, insert_base)
    
        

        
  

In [ ]:
exon_gene2 = ['TGTGTTGCATACTTTCTAAGGCGGCGGCTGCAGCAGCGGCTCCATCCAGCCCGTCAGCTCCTCCTGCAAGGCATGGCTGGCTACCTGAGTGAATCGGACTTTGTGATGGTGGAGGAGGGCTTCAGTACCCGAGACCTGCTGAAGGAACTCACTCTGGGGGCCTCACAGGCCACCACG',
  'GACGAGGTAGCTGCCTTCTTCGTGGCTGACCTGGGTGCCATAGTGAGGAAGCACTTTTGCTTTCTGAAGTGCCTGCCACGAGTCCGGCCCTTTTATGCTGTCAAGTGCAACAGCAGCCCAGGTGTGCTGAAGGTTCTGGCCCAGCTGGGGCTGGGCTTTAGCTGTGCCAACAAG',
  'GCAGAGATGGAGTTGGTCCAGCATATTGGAATCCCTGCCAGTAAGATCATCTGCGCCAACCCCTGTAAGCAAATTGCACAGATCAAATATGCTGCCAAGCATGGGATCCAGCTGCTGAGCTTTGACAATGAGATGGAGCTGGCAAAGGTGGTAAAGAGCCACCCCAGTGCCAA',
  'GATGGTTCTGTGCATTGCTACCGATGACTCCCACTCCCTGAGCTGCCTGAGCCTAAAGTTTGGAGTGTCACTGAAATCCTGCAGACACCTGCTTGAAAATGCGAAGAAGCACCATGTGGAGGTGGTGGGTGTGAG',
  'TTTTCACATTGGCAGTGGCTGTCCTGACCCTCAGGCCTATGCTCAGTCCATCGCAGACGCCCGGCTCGTGTTTGAAATGGGCACCGAGCTGGGTCACAAGATGCACGTTCTGGACCTTGGTGGTGGCTTCCCTGGCACAGAAGGGGCCAAAGTGAGATTTGAAGAG',
  'ATTGCTTCCGTGATCAACTCAGCCTTGGACCTGTACTTCCCAGAGGGCTGTGGCGTGGACATCTTTGCTGAGCTGGGGCGCTACTACGTGACCTCGGCCTTCACTGTGGCAGTCAGCATCATTGCCAAGAAGGAGGTTCTGCTAGACCAGCCTGGCAGGGAGG',
  'AGGAAAATGGTTCCACCTCCAAGACCATCGTGTACCACCTTGATGAGGGCGTGTATGGGATCTTCAACTCAGTCCTGTTTGACAACATCTGCCCTACCCCCATCCTGCAGAAG',
  'AAACCATCCACGGAGCAGCCCCTGTACAGCAGCAGCCTGTGGGGCCCGGCGGTTGATGGCTGTGATTGCGTGGCTGAGGGCCTGTGGCTGCCGCAACTACACGTAGGGGACTGGCTGGTCTTTGACAACATGGGCGCCTACACTGTGGGCATGGGTTCCCCCTTTTGGGGGACCCAGGCCTGCCACATCACCTATGCCATGTCCCGGGTGGCCTG',
  'GGAAGCGCTGCGAAGGCAGCTGATGGCTGCAGAACAGGAGGATGACGTGGAGGGTGTGTGCAAGCCTCTGTCCTGCGGCTGGGAGATCACAGACACCCTGTGCGTGGGCCCTGTCTTCACCCCAGCGAGCATCATGTGAGTGGGCCTCGTTCCCCCCGGAGAATCCCAGCGGGGCCTCAGAGATGCATCTGGGAGAGGTGGGGAAGATGGCAGGCAAGGGTACCCTTGGCCAGGACTCTGGTGCCCACCCTGCCACCCCCGCGCTCCACCTGCAGTGTTTCTGCCCTGTAAATAGGACCAGTCTTACACTCGCTGTAGTTCAAGTATGCAACATAAATCCTGTTCCTTCCA']

In [ ]:
exon_list_all_genes = extract_exons(chr_name)
exon_list_all_genes

In [ ]:
#gene2--works
fasta = pysam.FastaFile('/Users/Miko/Desktop/chromFa/' + chr_name +'.fa')

exon_list = [['33547778', '33547955'],
  ['33549554', '33549728'],
  ['33557650', '33557823'],
  ['33558882', '33559017'],
  ['33560148', '33560314'],
  ['33562307', '33562470'],
  ['33563667', '33563780'],
  ['33583502', '33583717'],
  ['33585644', '33585995']]
seq = ''
for index, exon in list(enumerate(exon_list)):
    start = exon[0]
    end = exon[1]
    seq += fasta.fetch('', int(start), int(end), chr_name)

print(seq)

In [ ]:
seq_list = extract_seq(chr_name, exon_list_all_genes)
seq_list

In [ ]:
seq_list_concat = extract_seq_concat(chr_name, exon_list_all_genes)
seq_list_concat